# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core import Dataset
import pandas as pd

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart_automl'

experiment=Experiment(ws, experiment_name)

In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute


compute_target = ComputeTarget(workspace=ws, name="ds11v2-cluster")
compute_target.wait_for_completion(show_output=True)

Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:



ds = Dataset.Tabular.from_delimited_files(path="https://archive.ics.uci.edu/ml/machine-learning-databases/00519/heart_failure_clinical_records_dataset.csv")
df = ds.to_pandas_dataframe(ds)



In [7]:
from decision_tree import clean
df = clean(df)
if "data" not in os.listdir():
    os.mkdir("./data")
df.to_csv('data/clean_heart_data.csv')

In [8]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='data',target_path='data')

Uploading an estimated of 3 files
Target already exists. Skipping upload for data/.amlignore
Target already exists. Skipping upload for data/.amlignore.amltmp
Uploading data/clean_heart_data.csv
Uploaded data/clean_heart_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_5d9b44d2b97647938461a8c664187d45

In [9]:
data = Dataset.Tabular.from_delimited_files(path=[(datastore,'data/clean_heart_data.csv')])

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:

automl_config = AutoMLConfig(
    experiment_timeout_minutes=60,
    task='classification',
    primary_metric='accuracy',
    training_data=data,
    label_column_name='DEATH_EVENT',
    validation_size=.1,
    test_size=.1,
    compute_target=compute_target,
    max_concurrent_iterations=2)

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart_automl,AutoML_b9bed883-c3eb-42e4-8f1f-db1407c9ac02,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [3]:
from azureml.core import Run
remote_run = Run(experiment,run_id="AutoML_b9bed883-c3eb-42e4-8f1f-db1407c9ac02")

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [4]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
import joblib

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)



Package:azureml-automl-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-core, training version:1.34.0, current version:1.33.0
Package:azureml-dataprep, training version:2.22.2, current version:2.20.1
Package:azureml-dataprep-rslex, training version:1.20.2, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-defaults, training version:1.34.0, current version:1.33.0
Package:azureml-interpret, training version:1.34.0, current version:1.33.0
Package:azureml-mlflow, training version:1.34.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.34.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.34.0, current version:1.33.0
Package:azureml-telemetry, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.34.0, current versi

Run(Experiment: heart_automl,
Id: AutoML_b9bed883-c3eb-42e4-8f1f-db1407c9ac02_1,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=False, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/jtgamino11/code/Users/jtgamino1/udacity_capstone')),
                ('MaxAbsScaler', MaxAbsScaler(copy=True)),
                ('XGBoostClassifier',
                 XGBoostClassifier(n_jobs=1, problem_info=ProblemInfo(
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=0, tree_method='auto'))],
         verbose=False)


In [19]:
best_run.download_file('outputs/model.pkl',output_file_path='outputs/best_automl_model.pkl')

In [22]:
#TODO: Save the best model
best_run.register_model(model_name="best_automl_model", model_path='./outputs/model.pkl')

Model(workspace=Workspace.create(name='SkynetML', subscription_id='17e15d04-6670-44fd-bf24-195e65650e31', resource_group='ML'), name=best_automl_model, id=best_automl_model:1, version=1, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service